In [4]:
### Step 1 Import Libraries

In [74]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [75]:
### Step 2 Importing dataset

In [76]:
df = pd.read_excel('a1_Dataset_10Percent.xlsx')
df=df.drop(['ID'],axis=1)

In [77]:
### Step 3 Explore missing values

In [78]:
df.isna().sum()

DemAffl            1085
DemAge             1508
DemClusterGroup     674
DemGender          2512
DemReg              465
DemTVReg            465
LoyalClass            0
LoyalSpend            0
LoyalTime           281
TargetBuy             0
dtype: int64

In [22]:
# filling missing values with mean/mode

df['DemAffl']=df['DemAffl'].fillna(df['DemAffl'].mode()[0])
df['DemAge']=df['DemAge'].fillna(df['DemAge'].mode()[0])
df['DemClusterGroup']=df['DemClusterGroup'].fillna(df['DemClusterGroup'].mode()[0])
df['DemGender']=df['DemGender'].fillna(df['DemGender'].mode()[0])
df['DemReg']=df['DemReg'].fillna(df['DemReg'].mode()[0])
df['DemTVReg']=df['DemTVReg'].fillna(df['DemTVReg'].mode()[0])
df['LoyalTime']=df['LoyalTime'].fillna(df['LoyalTime'].mean())

In [23]:
# explore missing values post missing value fix

print(df.isna().sum())

DemAffl            0
DemAge             0
DemClusterGroup    0
DemGender          0
DemReg             0
DemTVReg           0
LoyalClass         0
LoyalSpend         0
LoyalTime          0
TargetBuy          0
dtype: int64


In [24]:
#review the dataset
df.head(50)

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,TargetBuy
0,5.0,51.0,C,F,South East,London,Tin,0.01,5.00000,0
1,15.0,51.0,C,M,South East,London,Gold,8000.00,5.00000,1
2,8.0,51.0,C,M,Midlands,East,Tin,0.01,6.56467,1
3,8.0,28.0,C,F,South East,London,Tin,0.01,6.56467,1
4,14.0,67.0,C,F,South East,London,Tin,0.01,7.00000,0
5,8.0,65.0,C,F,South East,London,Silver,3000.00,3.00000,0
6,7.0,41.0,C,F,South East,London,Tin,0.01,7.00000,0
7,13.0,50.0,C,F,South East,London,Tin,0.01,5.00000,1
8,8.0,36.0,C,F,South East,London,Tin,0.01,9.00000,0
9,8.0,51.0,C,M,Midlands,Ulster,Silver,2000.00,1.00000,0


In [50]:
#converting the data to numeric values

number = LabelEncoder()

df['DemClusterGroup'] = number.fit_transform(df['DemClusterGroup'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['DemGender'] = number.fit_transform(df['DemGender'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['DemReg'] = number.fit_transform(df['DemReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['DemTVReg'] = number.fit_transform(df['DemTVReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['LoyalClass'] = number.fit_transform(df['LoyalClass'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)


{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'U': 6}
{'F': 0, 'M': 1, 'U': 2}
{'Midlands': 0, 'North': 1, 'Scottish': 2, 'South East': 3, 'South West': 4}
{'Border': 0, 'C Scotland': 1, 'East': 2, 'London': 3, 'Midlands': 4, 'N East': 5, 'N Scot': 6, 'N West': 7, 'S & S East': 8, 'S West': 9, 'Ulster': 10, 'Wales & West': 11, 'Yorkshire': 12}
{'Gold': 0, 'Platinum': 1, 'Silver': 2, 'Tin': 3}


In [58]:
#review the dataset
print(df.head(50))

    DemAffl  DemAge  DemClusterGroup  DemGender  DemReg  DemTVReg  LoyalClass  \
0       5.0    51.0                2          0       3         3           3   
1      15.0    51.0                2          1       3         3           0   
2       8.0    51.0                2          1       0         2           3   
3       8.0    28.0                2          0       3         3           3   
4      14.0    67.0                2          0       3         3           3   
5       8.0    65.0                2          0       3         3           2   
6       7.0    41.0                2          0       3         3           3   
7      13.0    50.0                2          0       3         3           3   
8       8.0    36.0                2          0       3         3           3   
9       8.0    51.0                2          1       0        10           2   
10      8.0    51.0                1          0       3         3           0   
11      8.0    51.0         

In [51]:
### Calculating VIF
def calc_vif(z):
    vif = pd.DataFrame()
    vif["variables"] = z.columns
    vif["VIF"] = [variance_inflation_factor(z.values, i) if is_numeric_dtype(z.iloc[:, i]) else np.nan for i in range(z.shape[1])]
    return(vif)

In [52]:
z = df.iloc[:,0:9]
calc_vif(z)

,variables,VIF
0,DemAffl,6.278630
1,DemAge,10.734656
2,DemClusterGroup,3.659632
3,DemGender,1.435472
4,DemReg,2.474645
5,DemTVReg,3.752279
6,LoyalClass,3.851766
7,LoyalSpend,1.863196
8,LoyalTime,3.153032


In [59]:
y = df.iloc[:,9].values
X = df.iloc[:,0:9].values

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [64]:

classifier =  LogisticRegression(max_iter=200)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [66]:
joblib.dump(classifier, './c2_Classifier_LoyalCustomers')

confusion_matrix(y_test,y_pred)

array([[3190,  177],
       [ 688,  390]], dtype=int64)

In [67]:
accuracy_score(y_test, y_pred)

0.8053993250843644

In [79]:
predictions = classifier.predict_proba(X_test)

# writing model output file

df_prediction_prob = pd.DataFrame(predictions, columns = ['prob_0', 'prob_1'])
df_test_dataset = pd.DataFrame(y_test,columns= ['Actual Outcome'])
df_x_test = pd.DataFrame(X_test)

dfx=pd.concat([df_x_test,df_test_dataset, df_prediction_prob], axis=1)

dfx.to_excel("c1_ModelOutput_10Percent.xlsx")

dfx.head()

,0,1,2,3,4,5,6,7,8,Actual Outcome,prob_0,prob_1
0,10.0,58.0,3.0,1.0,3.0,8.0,0.0,12632.66,4.0,0,0.862177,0.137823
1,10.0,51.0,2.0,0.0,3.0,8.0,0.0,6000.00,6.0,0,0.644901,0.355099
2,16.0,65.0,1.0,0.0,2.0,6.0,0.0,6053.35,7.0,0,0.485576,0.514424
3,5.0,60.0,3.0,0.0,3.0,3.0,2.0,5000.00,1.0,0,0.915096,0.084904
4,9.0,52.0,3.0,0.0,0.0,4.0,2.0,3500.00,6.0,0,0.703418,0.296582


In [80]:
df = pd.read_excel('a2_Dataset_90Percent.xlsx')
print(df.head())
# explore missing values

     ID  DemAffl  DemAge DemClusterGroup DemGender    DemReg      DemTVReg  \
0   140     10.0    76.0               C         U  Midlands  Wales & West   
1   620      4.0    49.0               D         U  Midlands  Wales & West   
2   868      5.0    70.0               D         F  Midlands  Wales & West   
3  1120     10.0    65.0               F         M  Midlands      Midlands   
4  2313     11.0    68.0               A         F  Midlands      Midlands   

  LoyalClass  LoyalSpend  LoyalTime  
0       Gold    16000.00        4.0  
1       Gold     6000.00        5.0  
2     Silver        0.02        8.0  
3        Tin        0.01        7.0  
4        Tin        0.01        8.0  


In [81]:
df.isna().sum()

ID                   0
DemAffl             48
DemAge              67
DemClusterGroup     28
DemGender          114
DemReg              18
DemTVReg            18
LoyalClass           0
LoyalSpend           0
LoyalTime           15
dtype: int64

In [82]:
# filling missing values with mean/mode*
df['DemAffl']=df['DemAffl'].fillna(df['DemAffl'].mode()[0])
df['DemAge']=df['DemAge'].fillna(df['DemAge'].mode()[0])
df['DemClusterGroup']=df['DemClusterGroup'].fillna(df['DemClusterGroup'].mode()[0])
df['DemGender']=df['DemGender'].fillna(df['DemGender'].mode()[0])
df['DemReg']=df['DemReg'].fillna(df['DemReg'].mode()[0])
df['DemTVReg']=df['DemTVReg'].fillna(df['DemTVReg'].mode()[0])
df['LoyalTime']=df['LoyalTime'].fillna(df['LoyalTime'].mean())

print(df.isna().sum())


ID                 0
DemAffl            0
DemAge             0
DemClusterGroup    0
DemGender          0
DemReg             0
DemTVReg           0
LoyalClass         0
LoyalSpend         0
LoyalTime          0
dtype: int64


In [87]:

number = LabelEncoder()

df['DemClusterGroup'] = number.fit_transform(df['DemClusterGroup'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['DemGender'] = number.fit_transform(df['DemGender'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['DemReg'] = number.fit_transform(df['DemReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['DemTVReg'] = number.fit_transform(df['DemTVReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

df['LoyalClass'] = number.fit_transform(df['LoyalClass'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

print(df.head())





{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}
{'0': 0, '1': 1, '2': 2}
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}
{'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12}
{'0': 0, '1': 1, '2': 2, '3': 3}
     ID  DemAffl  DemAge  DemClusterGroup  DemGender  DemReg  DemTVReg  \
0   140     10.0    76.0                2          2       0         6   
1   620      4.0    49.0                3          2       0         6   
2   868      5.0    70.0                3          0       0         6   
3  1120     10.0    65.0                5          1       0        10   
4  2313     11.0    68.0                0          0       0        10   

   LoyalClass  LoyalSpend  LoyalTime  
0           0    16000.00        4.0  
1           0     6000.00        5.0  
2           2        0.02        8.0  
3           3        0.01        7.0  
4           3        0.01        8.0  


In [86]:
x_fresh = df.iloc[:, 1:10].values

classifier = joblib.load('c2_Classifier_LoyalCustomers')

y_pred = classifier.predict(x_fresh)
print(y_pred)

predictions = classifier.predict_proba(x_fresh)
print(predictions)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0
 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 

In [84]:
# writing model output file
df_prediction_prob = pd.DataFrame(predictions, columns = ['prob_0', 'prob_1'])
dfx=pd.concat([df,df_prediction_prob], axis=1)
dfx.to_excel("d2_BuyProb_90Percent.xlsx")
dfx.head()

,ID,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,prob_0,prob_1
0,140,10.0,76.0,2,2,0,11,0,16000.00,4.0,0.975266,0.024734
1,620,4.0,49.0,3,2,0,11,0,6000.00,5.0,0.968690,0.031310
2,868,5.0,70.0,3,0,0,11,2,0.02,8.0,0.944575,0.055425
3,1120,10.0,65.0,5,1,0,4,3,0.01,7.0,0.890029,0.109971
4,2313,11.0,68.0,0,0,0,4,3,0.01,8.0,0.804335,0.195665
